# 1. Install and Import Dependencies

In [ ]:
# Execute this first
%pip install git+https://github.com/DLR-RM/stable-baselines3
# Then install the package with extras (gymnasium, atari, etc)
%pip install stable-baselines3[extra]

In [ ]:
# Install CUDA acceleration
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

In [ ]:
# You need to have tesseract installed: https://github.com/UB-Mannheim/tesseract/wiki
%pip install --upgrade pip
%pip install --upgrade Pillow
%pip install pytesseract

In [ ]:
# Screen capture and game control, using MSS instead of OpenCV
%pip install mss pydirectinput

In [ ]:
# MSS used for screen cap
from mss import mss
# Sending commands
import pydirectinput
# OpenCV allows us to do frame processing
import cv2
# Transformational framework
import numpy as np
# OCR for game over extraction
import pytesseract
# Visualize captured frames
from matplotlib import pyplot as plt
# For pauses
import time
# Environment components
from gymnasium import Env
from gymnasium.spaces import Box, Discrete

# 2. Build the Environment